<a href="https://colab.research.google.com/github/bydebby/python/blob/main/Data_Cleansing_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Cleansing - Pandas in a Notebook

In [ ]:
#Release: 1.2109.1901

In [ ]:
import pandas as pd
import numpy as np

# Create Dataframe

In [ ]:
#Dataframe from dictionary
f = {'name':["apples", "bananas", "cherries", "pears", "pears"],
    'quantity':[20, 20, 50, 30, 40],
    'price':[1000,500,750,900,950]}
df = pd.DataFrame(f)

#Add new rows
fnew = {'name':["grapes", "apples", "grapes", "oranges"],
        'quantity':[30,25,30,20],
        'price':[1200,800,1200,700]}

dfnew = pd.DataFrame(fnew,index=[5, 6, 7, 8])
df2 = df.append(dfnew)

#combine dataframes
colors = pd.DataFrame({'name':["apples", "bananas", "pears", "grapes", "cherries", "plum"], 
                       'color':['red', 'yellow', 'green', 'purple', 'red', 'purple']})
df3 = pd.merge(df2, colors, on='name', how='outer')
df3

,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple
7,grapes,30.0,1200.0,purple
8,oranges,20.0,700.0,NaN
9,plum,NaN,NaN,purple


###Add some rows to create 'dirty' data

In [ ]:
dirty = {'name':['Apples?', 'grapes', 'Lemons 2'],
        'quantity':['?', '?', np.nan],
        'price':[800,'?',700],
        'color':['green','?','yellow']}

df_dirty = pd.DataFrame(dirty,index=[10, 11, 12])
df_fruit = df3.append(df_dirty)
df_fruit

,name,quantity,price,color
0,apples,20,1000,red
1,apples,25,800,red
2,bananas,20,500,yellow
3,cherries,50,750,red
4,pears,30,900,green
5,pears,40,950,green
6,grapes,30,1200,purple
7,grapes,30,1200,purple
8,oranges,20,700,NaN
9,plum,NaN,NaN,purple


###Quick check the dataframe

In [ ]:
df_fruit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      13 non-null     object
 1   quantity  11 non-null     object
 2   price     12 non-null     object
 3   color     12 non-null     object
dtypes: object(4)
memory usage: 520.0+ bytes


In [ ]:
df_fruit.describe().transpose()

,count,unique,top,freq
name,13,9,grapes,3
quantity,11,6,30,3
price,12,9,700,2
color,12,5,green,3


#Missing values - Select

Missing values can cause to biased or even failure in training, because scikit-learn cannot process missing values.
What we need to do : 
1. Identify the missing values representations (`UNK`, `NaN`, `NULL`, `-9999`, `?`, etc.)
2. Decide on what to do with the missing values 
	- Remove all rows with any missing values (must be careful, since some columns may have too many missing values while other columns do not)
	- Remove the column with too many missing values
	- Impute values : can be done with simple method (impute with mean or mean value), with statistical methods, or other methods such as nearest neighbors value



Fungsi isna dapat digunakan untuk memeriksa apakah terdapat missing value, atau nilai NULL

In [ ]:
# isna
df_fruit.isna().any()

name        False
quantity     True
price        True
color        True
dtype: bool

In [ ]:
df_fruit.isna()

,name,quantity,price,color
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,True
9,False,True,True,False


In [ ]:
#Select the columns with NaN
df_fruit[df_fruit.columns[df_fruit.isna().any()]]

,quantity,price,color
0,20,1000,red
1,25,800,red
2,20,500,yellow
3,50,750,red
4,30,900,green
5,40,950,green
6,30,1200,purple
7,30,1200,purple
8,20,700,NaN
9,NaN,NaN,purple


###Invalid Values - Clean

Clean `name` column value, remove all non alphabet character.

In [ ]:
#select and print unique values to see if there's any invalid values
print(df_fruit['name'].unique()) 

['apples' 'bananas' 'cherries' 'pears' 'grapes' 'oranges' 'plum' 'Apples?'
 'Lemons 2']


In [ ]:
#remove all non-alphabetical character 
df_fruit['name'] = df_fruit.name.str.replace(r'[^a-zA-Z]\s?',r'')
print(df_fruit['name'].unique()) 

['apples' 'bananas' 'cherries' 'pears' 'grapes' 'oranges' 'plum' 'Apples'
 'Lemons']


In [ ]:
#change all to lowercase
df_fruit['name'] = df_fruit.name.str.lower()
df_fruit

,name,quantity,price,color
0,apples,20,1000,red
1,apples,25,800,red
2,bananas,20,500,yellow
3,cherries,50,750,red
4,pears,30,900,green
5,pears,40,950,green
6,grapes,30,1200,purple
7,grapes,30,1200,purple
8,oranges,20,700,NaN
9,plum,NaN,NaN,purple


###Convert column type

Set column with value = '?' as missing value (NaN, i.e. `np.nan`)

In [ ]:
for colname in ['quantity', 'price', 'color']:
  df_fruit.loc[df_fruit[colname] == '?', colname] = np.nan

df_fruit

,name,quantity,price,color
0,apples,20,1000,red
1,apples,25,800,red
2,bananas,20,500,yellow
3,cherries,50,750,red
4,pears,30,900,green
5,pears,40,950,green
6,grapes,30,1200,purple
7,grapes,30,1200,purple
8,oranges,20,700,NaN
9,plum,NaN,NaN,purple


In [ ]:
df_fruit["quantity"] = df_fruit["quantity"].astype(str).astype(float)
df_fruit["price"] = df_fruit["price"].astype(str).astype(float)
df_fruit.info()
df_fruit

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      13 non-null     object 
 1   quantity  9 non-null      float64
 2   price     11 non-null     float64
 3   color     11 non-null     object 
dtypes: float64(2), object(2)
memory usage: 520.0+ bytes


,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple
7,grapes,30.0,1200.0,purple
8,oranges,20.0,700.0,NaN
9,plum,NaN,NaN,purple


###Missing values - Remove

In [ ]:
df_fruit.dropna() #drop all rows with NaN in any column  

,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple
7,grapes,30.0,1200.0,purple


In [ ]:
df_fruit.dropna(subset = ['price', 'color'])

,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple
7,grapes,30.0,1200.0,purple
10,apples,NaN,800.0,green
12,lemons,NaN,700.0,yellow


In [ ]:
#let's update the dataframe
df_fruit.dropna(inplace=True)
df_fruit

,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple
7,grapes,30.0,1200.0,purple


###Duplicate rows 

In [ ]:
#dropping identical rows, keep the first occurence
df_fruit.drop_duplicates(inplace = True) #set inplace = True for in place modification
df_fruit

,name,quantity,price,color
0,apples,20.0,1000.0,red
1,apples,25.0,800.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
5,pears,40.0,950.0,green
6,grapes,30.0,1200.0,purple


In [ ]:
#We can also remove duplicate based on column
df_fruit.drop_duplicates("name")

,name,quantity,price,color
0,apples,20.0,1000.0,red
2,bananas,20.0,500.0,yellow
3,cherries,50.0,750.0,red
4,pears,30.0,900.0,green
6,grapes,30.0,1200.0,purple


Save to csv format

In [ ]:
df_fruit.to_csv('cleaned_data.csv', index=False)

# Revision History

Release: 1.2109.1901

* Code cleanup